In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical



In [2]:
df = pd.read_csv('D:\Work\huda\output.csv', low_memory=False)

df.head(1)

,id,student_name,student_email,student_gender,student_age,student_course,student_degree,course_id,enrolled_course
0,1,Harriet Welch,harriet.welch@outlook.com,Female,33,Business & Management,Bachelor Degree,"['MS-4430', 'BM-7543', 'MA-7693', 'CS-8976', '...",MS-4430


In [3]:
import ast

df.course_id = df.course_id.apply(ast.literal_eval)

In [4]:
from sklearn.preprocessing import MultiLabelBinarizer

s = df['course_id']

mlb = MultiLabelBinarizer()

course_dummies = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df.index)

In [5]:
df_basic = df[['id', 'student_gender', 'student_age', 'student_course', 'student_degree', 'course_id', 'enrolled_course']]
df_new = pd.concat([df_basic, course_dummies], axis=1)
df_new = pd.get_dummies(df_new, columns=['student_gender', 'student_course', 'student_degree'])
df_new.head(1)

,id,student_age,course_id,enrolled_course,BA-5120,BA-6633,BA-7701,BA-8090,BA-8551,BA-9801,...,student_course_Computer Science and Arts,student_course_Education,student_course_Engineering,student_course_Law,student_course_Medicine,student_course_Nursing,student_course_Science,student_degree_Bachelor Degree,student_degree_Doctorate Degree,student_degree_Master Degree
0,1,33,"[MS-4430, BM-7543, MA-7693, CS-8976, BZ-6600, ...",MS-4430,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
from sklearn.preprocessing import LabelEncoder

X = df_new.drop(['course_id', 'id', 'enrolled_course'], axis=1)

encoder = LabelEncoder()
Y = pd.get_dummies(encoder.fit_transform(df_new.enrolled_course)).values
y = df_new.enrolled_course

In [7]:
encoder.classes_

array(['BA-5120', 'BA-6633', 'BA-7701', 'BA-8090', 'BA-8551', 'BA-9801',
       'BB-1629', 'BB-5511', 'BC-0025', 'BC-4400', 'BC-5759', 'BC-7484',
       'BD-1110', 'BD-1199', 'BD-4000', 'BD-4878', 'BD-7375', 'BE-2983',
       'BE-3311', 'BE-5421', 'BF-3399', 'BF-7263', 'BH-7020', 'BL-2244',
       'BL-7247', 'BM-0033', 'BM-0203', 'BM-2211', 'BM-2324', 'BM-7543',
       'BM-7565', 'BN-2277', 'BN-5950', 'BN-7775', 'BQ-5533', 'BR-1790',
       'BR-5867', 'BS-0318', 'BS-1818', 'BS-6070', 'BS-7230', 'BS-8990',
       'BT-1144', 'BV-0035', 'BV-1199', 'BV-4565', 'BZ-6600', 'CA-2090',
       'CD-8321', 'CP-2047', 'CS-2314', 'CS-8976', 'DA-4499', 'DA-7979',
       'DC-4639', 'DC-5050', 'DC-8010', 'DD-7565', 'DE-0102', 'DE-2211',
       'DE-2910', 'DL-1001', 'DL-1090', 'DM-0099', 'DM-7571', 'DQ-3535',
       'DS-4771', 'DS-5456', 'DS-7771', 'DT-6020', 'DW-2456', 'DX-3738',
       'EN-6721', 'MA-1254', 'MA-2266', 'MA-3601', 'MA-6050', 'MA-6112',
       'MA-7693', 'MA-8221', 'MB-0501', 'MB-7766', 

In [14]:
len(encoder.classes_)

131

In [8]:
Y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=21)
_, _, _, _y_test = train_test_split(X, y, test_size=0.25, random_state=21)

In [10]:
# pretrained_model = tf.keras.applications.ResNet50(include_top=False)
from tensorflow.keras import regularizers

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(147,)),
    # tf.keras.layers.Dropout(0.5),
    # tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    # tf.keras.layers.Dropout(0.5),
    # tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    # tf.keras.layers.Dropout(0.5),
    # pretrained_model,
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(131, activation='softmax')
  ])
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [16]:
len(X_train.columns)

147

In [73]:
model.fit(X_train, y_train, batch_size=50, epochs=10)

Epoch 1/10
150/150 [==============================] - 1s 2ms/step - loss: 5.1577 - accuracy: 0.0107
Epoch 2/10
150/150 [==============================] - 0s 1ms/step - loss: 4.7935 - accuracy: 0.0327
Epoch 3/10
150/150 [==============================] - 0s 2ms/step - loss: 4.6716 - accuracy: 0.0672
Epoch 4/10
150/150 [==============================] - 0s 2ms/step - loss: 4.5546 - accuracy: 0.1111
Epoch 5/10
150/150 [==============================] - 0s 1ms/step - loss: 4.4377 - accuracy: 0.1480
Epoch 6/10
150/150 [==============================] - 0s 1ms/step - loss: 4.3262 - accuracy: 0.1647
Epoch 7/10
150/150 [==============================] - 0s 2ms/step - loss: 4.2138 - accuracy: 0.1857
Epoch 8/10
150/150 [==============================] - 0s 2ms/step - loss: 4.0982 - accuracy: 0.2031
Epoch 9/10
150/150 [==============================] - 0s 2ms/step - loss: 3.9801 - accuracy: 0.2129
Epoch 10/10
150/150 [==============================] - 0s 2ms/step - loss: 3.8706 - accuracy: 0.2223

In [74]:
y_pred = model.predict(X_test)
y_pred_class = np.argmax(model.predict(X_test), axis=-1)
y_pred_class

79/79 [==============================] - 0s 1ms/step


array([52, 86, 15, ..., 37, 76, 48], dtype=int64)

In [75]:
print(y_pred_class)

[52 86 15 ... 37 76 48]


In [83]:
y_pred_class_labels = encoder.inverse_transform(y_pred_class)
y_pred_class_labels

array(['DA-4499', 'MC-3999', 'BD-4878', ..., 'BS-0318', 'MA-6050',
       'CD-8321'], dtype=object)

In [84]:
from sklearn.metrics import classification_report
report = pd.DataFrame(classification_report(_y_test, y_pred_class_labels, output_dict=True)).transpose()

d:\Work\venvs\ds\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Work\venvs\ds\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Work\venvs\ds\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [86]:
report.to_csv('dl_cm.csv')